### demand_train.csv
|字段|说明|
|-|-|
|unit||

In [1]:
from pandas import datetime
from matplotlib import pyplot
from pandas.plotting import autocorrelation_plot
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA


In [2]:
using_hist = pd.read_csv("../data/Dataset/demand_train_A.csv")  # 历史需求
using_future = pd.read_csv("../data/Dataset/demand_test_A.csv")  # 未来信息
inventory = pd.read_csv("../data/Dataset/inventory_info_A.csv")
last_dt = pd.to_datetime("20210301")
start_dt = pd.to_datetime("20210302")
end_dt = pd.to_datetime("20210607")
lead_time = 14

using_hist["ts"] = using_hist["ts"].apply(lambda x: pd.to_datetime(x))
using_future["ts"] = using_future["ts"].apply(lambda x: pd.to_datetime(x))
qty_using = pd.concat([using_hist, using_future])
date_list = pd.date_range(start=start_dt, end=end_dt)
unit_list = using_future["unit"].unique()
res = pd.DataFrame(columns=["unit", "ts", "qty"])

In [7]:
using_future.head()

,Unnamed: 0,unit,ts,qty,geography_level,geography,product_level,product
0,0,0efede250ca3d05f9d4cc3609242d804,2021-03-02,3437.199978,geography_level_3,7b61250107b0ab94a35b9b80168df40d,product_level_2,1de8e43efce0cb9161ac2093f75166c8
1,1,fbb83aefc6f5d6f6bc22ae3ee757d327,2021-03-02,34.067925,geography_level_3,8b2bbea7dd24f1427bd37e74936047c7,product_level_2,646f301a50470456f3e7793f6b16ca76
2,2,392aaa20e70b4d7539cc7a2e09562521,2021-03-02,34.856490,geography_level_3,e77875a9e01c74e943c9ed3c346d1034,product_level_2,b04c1602cb78c1c4cf72dd7c0ce7d837
3,3,2effa036807329a88056093fabb07ce6,2021-03-02,36677.666667,geography_level_3,52b9c00ab15c967f2f49c5605800fef5,product_level_2,93a97f63f0a573483d49c58dc02c1341
4,4,7dc25ea61b4d47f7de6c7a8d8d559487,2021-03-02,56688.333333,geography_level_3,52b9c00ab15c967f2f49c5605800fef5,product_level_2,6c46c9c5d3ef936b4c8d6d8f9851e837


### 提供每个unit的历史数据，用于评估预测模型的性能．

每个商品利用测试集时间做测试．

In [8]:
demand_dict = {}

for chunk in qty_using.groupby("unit"):  # 按unit分组计算
    unit = chunk[0]
    demand = chunk[1]
    demand.sort_values("ts", inplace=True, ascending=True)

    # 计算净需求量 = 毛需求－(现有库存－现有库存已分配量)－在途量＋安全库存
    demand["diff"] = demand["qty"].diff().values
    # delete the first row
    demand = demand[1:]
    demand_dict[unit] = demand[(demand["unit"] == unit)]

/home/amax/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/amax/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
def mae_metric(_pred, _true):
    return np.mean(np.abs(_pred - _true))

In [16]:
from scipy import linalg
print(linalg.lapack.dgetrf([np.nan]))
print(linalg.lapack.dgetrf([np.inf]))

(array([nan]), array([0], dtype=int32), 0)
(array([inf]), array([0], dtype=int32), 0)


In [29]:

mae = []
count = 0
len_error = 0
for date in date_list:
    #按每日净需求与每日补货到达更新状态，并判断补货量
    if date.dayofweek == 0:
        for unit in unit_list:
            demand = demand_dict[unit]
            demand_hist = demand[demand["ts"] < date]
            true = demand[demand["ts"] >= date]['diff'][:lead_time]

            if len(true) < 14:
                len_error += 1
            else:
                try:
                    series = pd.Series(np.array(demand_hist["diff"].values[-3 * 7:]))
                    model = ARIMA(series, order=(7, 0, 0))
                    model_fit = model.fit()
                    output = model_fit.forecast(steps=14)
                    mae.append(mae_metric(output.values, true))
                except:
                    count += 1
                    continue
                    
print('****************************')
print(np.mean(np.array(mae)))

/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retva

/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-

/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-

/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-

/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-

...

/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-

/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-

/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-

/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  ConvergenceWarning)
/home/amax/anaconda3/lib/python3.6/site-

****************************
28.952584900412


### ARIMA 使用历史2周的预测的MAE为45.57 净需求量，并非QTY
### ARIMA 使用历史3周的数据进行预测 预测ＭAE 28.95

In [25]:
mae = []
count = 0
len_error = 0
for date in date_list:
    #按每日净需求与每日补货到达更新状态，并判断补货量
    if date.dayofweek == 0:
        for unit in unit_list:
            demand = demand_dict[unit]
            demand_hist = demand[demand["ts"] < date]
            true = demand[demand["ts"] >= date]['diff'][:lead_time]

            if len(true) < 14:
                len_error += 1
            else:
                demand_average = np.mean(demand_hist["diff"].values[-3 * lead_time:])
                output = np.array(demand_average * lead_time)
                mae.append(mae_metric(output, true))
                    
print('****************************')
print(np.mean(np.array(mae)))

****************************
111.33440146236914


### MEAN 预测的MAE为111.33

In [3]:
submission['diff_1'] = submission['qty'] - submission['yesterday_qty']
submission['qty'] = submission['diff_1']

submission['shift_14']=submission.groupby('unit')['qty'].shift(-14).fillna(0).reset_index().sort_index().set_index('index')
submission = submission[['unit','ts','shift_14']].rename(columns={'shift_14':'qty'})


#按照7天聚合
submission['dt'] = pd.to_datetime(submission['ts'])
submission['weekofyear'] = submission['dt'].dt.weekofyear
submission['year'] = submission['dt'].dt.year
submission_week = submission.copy()
submission_week = submission_week.groupby(['weekofyear','year','unit'],as_index=False).sum()
submission_week['sum_qty'] = submission_week['qty']
submission = pd.merge(submission_week,submission,on = ['weekofyear','year','unit'])
submission['dayofweek'] = submission['dt'].dt.dayofweek
submission = submission[submission['dayofweek']==0]
submission = submission[['unit','ts','sum_qty']].rename(columns={'sum_qty':'qty'})


init_inventory = inventory_info_A.set_index(['unit'])['qty'].to_dict()
def consume_init_inventory(arr,init_val):
    remain = init_val
    i = 0
    while remain>0 and i<len(arr):
        arr[i] = max(0,arr[i]-remain)
        remain -= arr[i]
        i+=1
    return arr

r = []
for i,group in submission.groupby('unit'):

    unit = group['unit'].values[0]
    init_val = init_inventory[unit]
    
    group = group.sort_values(by='ts')
    qty_list = group['qty'].values
    qty_list = consume_init_inventory(qty_list,init_val)
    group['qty'] = qty_list
    r.append(group)

submission = pd.concat(r)


In [7]:
submission['qty'] = submission['qty'].astype(int)

In [8]:
submission.head()

,unit,ts,qty
3792,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-03-08,0
8216,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-03-15,0
12640,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-03-22,0
17064,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-03-29,0
21488,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-04-05,0


In [9]:
import datetime

submission.to_csv(("../submit/submit_"+datetime.datetime.now().strftime('%Y%m%d_%H%M%S') + ".csv"), index=False)